In [1]:
import numpy as np
import pandas as pd


import re
import time

In [2]:
data = pd.read_csv('../adult/categorically_numbered.csv', names=["age","workclass","fnlwgt","education-num",
                                                                 "marital-status","occupation","race","sex",
                                                                 "capital-gain","capital-loss","hours-per-week","native-country","salary"])

In [3]:
data

,age,workclass,fnlwgt,education-num,marital-status,occupation,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,0,77516,12,6,6,0,0,2174,0,40,0,0
1,50,4,83311,12,0,7,0,0,0,0,13,0,0
2,38,3,215646,8,3,3,0,0,0,0,40,0,0
3,53,3,234721,6,0,3,1,0,0,0,40,0,0
4,28,3,338409,12,0,8,1,1,0,0,40,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30157,27,3,257302,11,0,10,0,1,0,0,38,0,0
30158,40,3,154374,8,0,2,0,0,0,0,40,0,1
30159,58,3,151910,8,5,6,0,1,0,0,40,0,0
30160,22,3,201490,8,6,6,0,0,0,0,20,0,0


In [4]:
anon_data = pd.read_csv('anon_data/datafly/datafly2.csv', names=["age","workclass","fnlwgt","education-num",
                                                                 "marital-status","occupation","race","sex",
                                                                 "capital-gain","capital-loss","hours-per-week","native-country","salary"])

In [5]:
anon_data

,age,workclass,fnlwgt,education-num,marital-status,occupation,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,[0:50),[0:6],[0:1500000),[9:15],[0:6],[0:13],[0:0],[0.0],[0:100000),[0:1),[0:100],[0:39],0
1,[50:100),[0:6],[0:1500000),[9:15],[0:6],[0:13],[0:0],[0.0],[0:100000),[0:1),[0:100],[0:39],0
2,[0:50),[0:6],[0:1500000),[0:8],[0:6],[0:13],[0:0],[0.0],[0:100000),[0:1),[0:100],[0:39],0
3,[50:100),[0:6],[0:1500000),[0:8],[0:6],[0:13],[1:4],[0.0],[0:100000),[0:1),[0:100],[0:39],0
4,[0:50),[0:6],[0:1500000),[9:15],[0:6],[0:13],[1:4],[1.0],[0:100000),[0:1),[0:100],[0:39],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30157,[0:50),[0:6],[0:1500000),[9:15],[0:6],[0:13],[0:0],[1.0],[0:100000),[0:1),[0:100],[0:39],0
30158,[0:50),[0:6],[0:1500000),[0:8],[0:6],[0:13],[0:0],[0.0],[0:100000),[0:1),[0:100],[0:39],1
30159,[50:100),[0:6],[0:1500000),[0:8],[0:6],[0:13],[0:0],[1.0],[0:100000),[0:1),[0:100],[0:39],0
30160,[0:50),[0:6],[0:1500000),[0:8],[0:6],[0:13],[0:0],[0.0],[0:100000),[0:1),[0:100],[0:39],0


In [22]:
def parse_range(x):
    if ':' not in x:
        val = int(re.split('\.', x[1:-1])[0])
        return val, val
    
    vals = re.split(':', x[1:-1])
    low = int(vals[0]) if x[0] == '[' else int(vals[0]) + 1 
    high = int(vals[1]) if x[-1] == ']' else int(vals[1]) - 1 
    return low, high

def conditional_prob(col, val, bounds):
    lo,hi = bounds
    subset = data[(data[col] >= lo) & (data[col] <= hi)][col]
    print(len(subset))
    counts = subset.value_counts()
    num = counts[val] if val in counts else 0
    denom = np.sum([counts[x] for x in subset])
    return num/denom

In [23]:
def H(col, ind):
    lo, hi = parse_range(anon_data[col][ind])
    counts = data[col].value_counts()
    seen = {}
    probs = []
    for c in counts.index:
        if counts[c] in seen:
            probs.append(seen[counts[c]])
        else:
            p = conditional_prob(col, c, (lo,hi))
            probs.append(p)
            seen[counts[c]] = p
        
    entropies = [np.log2(x)*x if x else 0 for x in probs]
    return -1 * np.sum(entropies)

tot_entr = 0
for col in anon_data.columns[:-1]:
    print(col)
    print("Set of values found: ", anon_data[col].unique())
    seen = {}
    col_entr = 0
    for ind in range(0,len(anon_data)):
        if ind in anon_data.index:
            cell = anon_data[col][ind]
            if cell in seen:
                col_entr += seen[cell]
            else:
                cell_entr = H(col, ind)
                print(f"new cell added:{cell} with entropy {cell_entr}")
                col_entr += cell_entr
                seen[cell] = cell_entr
    print("column entropy:", col_entr)
    print("##############################")
    tot_entr += col_entr
print(tot_entr)

age
Set of values found:  ['[0:50)' '[50:100)' '[0:100)']
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
23840
new cell added:[0:50) with entropy 0.019223813841350447
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
6251
new cell added:[50:100) with entropy 0.04013420371529483
30091
30091
30091
30091
30091
30091
30091
30091
30091
30091
30091
30091
30091
30091
30091
30091
30091
30091
30091


KeyboardInterrupt: 